## 实现了kmeans及kmeans++,并通过hct66数据集与mnist数据集分别进行了验证

In [7]:
from hct66 import *
from kmeans import *

## 一、hct66数据集

### 加载数据

In [2]:
image_data,image_label = generate_data()
data_num = len(image_data)
data = torch.empty((data_num, 6)).float()
for i in range(data_num):
    data[i] = get_feature(image_data[i]).view(6)
    
print(data.shape)


torch.Size([10, 6])


### kmeans方法聚类结果

In [7]:
# 创建模型
kmeans = KMeans(n_clusters=3, init_random=True)
# 开始训练
kmeans.fit(data)
# 开始预测
pred = kmeans.predict(data)
for i in range(data_num):
    print("数字:{}聚类结果:{}".format(i, pred[i]))

聚类中心点偏移程度:1.9930435419082642
聚类中心点偏移程度:0.7731608748435974
聚类中心点偏移程度:0.0
数字:0聚类结果:2
数字:1聚类结果:0
数字:2聚类结果:2
数字:3聚类结果:0
数字:4聚类结果:2
数字:5聚类结果:1
数字:6聚类结果:1
数字:7聚类结果:0
数字:8聚类结果:2
数字:9聚类结果:2


### kmeans++方法聚类结果

In [8]:
# 创建模型
kmeans = KMeans(n_clusters=3, init_random=False)
# 开始训练
kmeans.fit(data)
# 开始预测
pred = kmeans.predict(data)
for i in range(data_num):
    print("数字:{}聚类结果:{}".format(i, pred[i]))

聚类中心点偏移程度:1.60173499584198
聚类中心点偏移程度:0.0
数字:0聚类结果:2
数字:1聚类结果:0
数字:2聚类结果:0
数字:3聚类结果:0
数字:4聚类结果:2
数字:5聚类结果:1
数字:6聚类结果:1
数字:7聚类结果:0
数字:8聚类结果:0
数字:9聚类结果:2


### 结论：
#### 两种方法只有数字8的聚类结果不一样，其他都一样

## 二、mnist数据集

### 加载数据集

In [8]:
import sys
sys.path.append('../week_4/')
from data import load_mnist

train_imgs, train_labels = load_mnist('../week_4/mnist', kind='train')
test_imgs, test_labels = load_mnist('../week_4/mnist', kind='t10k')

# 选取训练集数据样本量为1000，测试集为100
train_imgs, train_labels = train_imgs[:1000], train_labels[:1000]
test_imgs, test_labels = test_imgs[:100], test_labels[:100]

### 数据预处理

In [9]:
from sklearn.decomposition import PCA
from sklearn import preprocessing

# 数据标准化
zscore = preprocessing.StandardScaler()                # zscore标准化方式
train_imgs = zscore.fit_transform(train_imgs)
test_imgs = zscore.transform(test_imgs)                # 测试集与训练集保持一致的zscore方式
# PCA降维
pca = PCA(n_components=30)  
train_features = pca.fit_transform(train_imgs)    # 训练集pca降维
test_features = pca.transform(test_imgs)          # 测试集pca降维

### 评估准确率接口

In [35]:
def get_cluster_idx(labels_1, labels_2):
    # labels_1:数据对应的数字
    # labels_2: 模型中聚类得到的标签
    cluster_idx = []
    for i in range(10):
        idx = np.argwhere(labels_1 == i)
        tmp = labels_2[idx]
        tmp = np.squeeze(tmp)
        cluster_idx.append(np.argmax(np.bincount(tmp)))
    return cluster_idx

def get_acc(test_labels, test_pred, cluster_idx):
    acc = []
    for i in range(10):
        idx = np.argwhere(test_labels == i)
        tmp = test_pred[idx]
        tmp = np.squeeze(tmp)
        right_idx = np.argwhere(tmp == cluster_idx[i])
        a = right_idx.shape[0] / tmp.shape[0] * 100
        acc.append(a)
        print('数字:{}预测准确率:{}'.format(i, a))
    return acc

### kmeans方法聚类结果

In [36]:
kmeans = KMeans(n_clusters=10, init_random=True)
# 开始训练
kmeans.fit(torch.from_numpy(train_features))
# 开始预测
pred = kmeans.predict(torch.from_numpy(test_features))
data_num = test_features.shape[0]
print("预测结果...")
for i in range(data_num):
    print("数字:{}聚类结果:{}".format(test_labels[i], pred[i]))
    
print("准确率统计...")
cluster_idx = get_cluster_idx(train_labels, kmeans.labels.numpy())
acc = get_acc(test_labels, pred.numpy(), cluster_idx)

聚类中心点偏移程度:29.607247623960113
聚类中心点偏移程度:10.156621932983398
聚类中心点偏移程度:5.89716911315918
聚类中心点偏移程度:7.949558734893799
聚类中心点偏移程度:6.388545989990234
聚类中心点偏移程度:4.974133491516113
聚类中心点偏移程度:4.173639297485352
聚类中心点偏移程度:2.0072414875030518
聚类中心点偏移程度:2.2811224460601807
聚类中心点偏移程度:2.1374905109405518
聚类中心点偏移程度:1.5519706010818481
聚类中心点偏移程度:2.4223246574401855
聚类中心点偏移程度:1.5457338094711304
聚类中心点偏移程度:0.6535611152648926
聚类中心点偏移程度:0.8288869261741638
聚类中心点偏移程度:0.0
预测结果...
数字:7聚类结果:5
数字:2聚类结果:8
数字:1聚类结果:9
数字:0聚类结果:2
数字:4聚类结果:1
数字:1聚类结果:9
数字:4聚类结果:1
数字:9聚类结果:8
数字:5聚类结果:6
数字:9聚类结果:5
数字:0聚类结果:2
数字:6聚类结果:1
数字:9聚类结果:1
数字:0聚类结果:1
数字:1聚类结果:9
数字:5聚类结果:8
数字:9聚类结果:1
数字:7聚类结果:1
数字:3聚类结果:8
数字:4聚类结果:1
数字:9聚类结果:5
数字:6聚类结果:0
数字:6聚类结果:1
数字:5聚类结果:6
数字:4聚类结果:1
数字:0聚类结果:7
数字:7聚类结果:1
数字:4聚类结果:1
数字:0聚类结果:2
数字:1聚类结果:9
数字:3聚类结果:1
数字:1聚类结果:9
数字:3聚类结果:8
数字:4聚类结果:6
数字:7聚类结果:5
数字:2聚类结果:4
数字:7聚类结果:1
数字:1聚类结果:9
数字:2聚类结果:9
数字:1聚类结果:9
数字:1聚类结果:9
数字:7聚类结果:1
数字:4聚类结果:1
数字:2聚类结果:9
数字:3聚类结果:9
数字:5聚类结果:9
数字:1聚类结果:1
数字:2聚类结果:9
数字:4聚类结果:1
数字:4聚类结果:1

### kmeans++聚类结果

In [37]:
kmeans_pp = KMeans(n_clusters=10, init_random=False)
# 开始训练
kmeans_pp.fit(torch.from_numpy(train_features))
# 开始预测
pred_pp = kmeans.predict(torch.from_numpy(test_features))
data_num = test_features.shape[0]
print("预测结果...")
for i in range(data_num):
    print("数字:{}聚类结果:{}".format(test_labels[i], pred_pp[i]))
    
print("准确率统计...")
cluster_idx_pp = get_cluster_idx(train_labels, kmeans_pp.labels.numpy())
acc_pp = get_acc(test_labels, pred_pp.numpy(), cluster_idx_pp)

聚类中心点偏移程度:46.747520446777344
聚类中心点偏移程度:11.444483757019043
聚类中心点偏移程度:10.786985397338867
聚类中心点偏移程度:7.642075061798096
聚类中心点偏移程度:4.480526924133301
聚类中心点偏移程度:3.0219664573669434
聚类中心点偏移程度:2.218599557876587
聚类中心点偏移程度:2.207875967025757
聚类中心点偏移程度:1.4722528457641602
聚类中心点偏移程度:0.6805112957954407
聚类中心点偏移程度:2.809535026550293
聚类中心点偏移程度:0.1043250560760498
聚类中心点偏移程度:0.1549130529165268
聚类中心点偏移程度:0.19813208281993866
聚类中心点偏移程度:0.2290148138999939
聚类中心点偏移程度:0.15154777467250824
聚类中心点偏移程度:0.1884559690952301
聚类中心点偏移程度:0.13066843152046204
聚类中心点偏移程度:0.10175049304962158
聚类中心点偏移程度:0.30542436242103577
聚类中心点偏移程度:0.11306161433458328
聚类中心点偏移程度:0.18460607528686523
聚类中心点偏移程度:0.15681540966033936
聚类中心点偏移程度:0.10637231171131134
聚类中心点偏移程度:0.09891415387392044
聚类中心点偏移程度:0.10511888563632965
聚类中心点偏移程度:0.2369125634431839
聚类中心点偏移程度:0.2756730914115906
聚类中心点偏移程度:0.06563685834407806
聚类中心点偏移程度:0.04991510137915611
聚类中心点偏移程度:0.0
预测结果...
数字:7聚类结果:5
数字:2聚类结果:8
数字:1聚类结果:9
数字:0聚类结果:2
数字:4聚类结果:1
数字:1聚类结果:9
数字:4聚类结果:1
数字:9聚类结果:8
数字:5聚类结果:6
数

### 总结  
#### 对于mnist环节，准确率低是因为采用了很小的数据集，旨在跑通代码，实现功能，尚未使用全部数据对模型调优